In [1]:
import pandas as pd
import numpy as np
import os
import random
import progressbar
import multiprocessing
import pickle
import matplotlib.pyplot as plt

data_path = 'gdrive/My Drive/Summer Research/hmp2-data-stanford/Preprocessed/'

data_choices = {
    data_path: '',
    data_path+'Denoised/': 'Denoised',
    data_path+'WT Domain/': 'T*S',
    data_path+'Normalized/': 'Normalized',
    data_path+'Normalized/'+'Denoised/': 'Denoised Normalized',
    data_path+'Normalized/'+'WT Domain/': 'T*S Normalized',
}

from xgboost import XGBClassifier
from sklearn.metrics import (plot_confusion_matrix, plot_precision_recall_curve,
                             plot_roc_curve, auc)
from sklearn.model_selection import cross_validate, StratifiedKFold

import seaborn as sb
from statistics import mean, stdev

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Oversampling and undersampling

In [2]:
def resampling(args):
  if args == 'SMOTEENN':
    resampler = SMOTEENN(n_jobs=-1)
  elif args == 'SMOTETomek':
    resampler = SMOTETomek(n_jobs=-1)
  
  return resampler

XGBoost model

In [3]:
def XGBModel(X, y, cv):
  model = XGBClassifier()
  #K-fold Cross Validation
  scores = cross_validate(model, X, y, cv=cv, scoring=('accuracy', 'balanced_accuracy', 'precision', 'recall', 'roc_auc', 'f1'), n_jobs=-1, verbose=0, return_estimator=True)
  return scores

In [4]:
def metrics(scores, X, y, cv, resampling_method, data_choice, hmp_data_choice):
  dir = 'gdrive/My Drive/Summer Research/Figures/HMP Data/XGBoost/'
  file_name = hmp_data_choice + '/' + data_choice
  try:
    os.makedirs(dir+hmp_data_choice+'/')
  except FileExistsError:
    pass
  rem_list = ['estimator', 'fit_time', 'score_time']
  csv_scores = dict([(key, val) for key, val in 
           scores.items() if key not in rem_list])
  df = pd.DataFrame.from_dict(csv_scores)
  df.to_csv(dir+file_name+' Metrics.csv', index=False)

  #TODO: generate PR, ROC, Confusion matrix graphs
  tprs = []
  aucs = []
  mean_fpr = np.linspace(0, 1, 100)

  cm = np.zeros((4,10))

  fig, ax = plt.subplots(figsize=(10,10))
  fig2, ax2 = plt.subplots(figsize=(10,10))
  fig3, ax3 = plt.subplots(figsize=(10,10))
  fig4, ax4 = plt.subplots(figsize=(10,10))
  for i, (train, test) in enumerate(cv.split(X, y)):
    viz = plot_roc_curve(scores['estimator'][i], X[test], y[test],
                         name='ROC fold {}'.format(i),
                         alpha=0.3, lw=1, ax=ax)
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

    p = plot_precision_recall_curve(scores['estimator'][i], X[test],
                                     y[test], name='P v. R fold {}'.format(i),
                                     alpha=0.5, lw=1.5, ax=ax2)
    
    c = plot_confusion_matrix(scores['estimator'][i], X[test], y[test],
                              normalize='all', ax=ax4)
    cm[:,i] = np.array(c.confusion_matrix).reshape(4,)
  plt.close(fig=fig4)
  #ROC Curve
  ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
        label='Chance', alpha=.8)
  mean_tpr = np.mean(tprs, axis=0)
  mean_tpr[-1] = 1.0
  mean_auc = auc(mean_fpr, mean_tpr)
  std_auc = np.std(aucs)
  ax.plot(mean_fpr, mean_tpr, color='b',
          label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
          lw=2, alpha=.8)

  std_tpr = np.std(tprs, axis=0)
  tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
  tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
  ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                  label=r'$\pm$ 1 std. dev.')

  ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
        title="ROC Curve")
  ax.legend(loc="lower right")
  fig.savefig(dir+file_name+' ROC.png', bbox_inches='tight')
  plt.close(fig=fig)
  #PR Curve
  ax2.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
        title="Precision v. Recall Curve")
  ax2.legend(loc="lower left")
  fig2.savefig(dir+file_name+' PR.png', bbox_inches='tight')
  plt.close(fig=fig2)
  #Confusion Matrix
  c1, c2, c3, c4 = cm[0,:], cm[1,:], cm[2,:], cm[3,:]
  means = np.array([[mean(c1), mean(c2)],[mean(c3), mean(c4)]])
  stds = np.array([[stdev(c1), stdev(c2)],[stdev(c3), stdev(c4)]])
  labels = np.array([["{:.2%} $\pm$ {:.2%}".format(mean(c1), stdev(c1)),
                      "{:.2%} $\pm$ {:.2%}".format(mean(c2), stdev(c2))],
                     ["{:.2%} $\pm$ {:.2%}".format(mean(c3), stdev(c3)),
                      "{:.2%} $\pm$ {:.2%}".format(mean(c4), stdev(c4))]])
  plt.figure(figsize=(12,8))
  g = sb.heatmap(100*means, fmt='', annot=labels, cmap='Greens',
                 xticklabels=['Predicted IS', 'Predicted IR'],
                 yticklabels=['IS', 'IR'], ax=ax3, cbar_kws={'format': '%.0f%%'})
  g.set_yticklabels(labels=g.get_yticklabels(), va='center')
  g.set_title('Confusion Matrix')
  fig3.savefig(dir+file_name+' Confusion Matrix.png', bbox_inches='tight')
  plt.close(fig=fig3)
  plt.close('all')

In [5]:
def run_model(data_choice, hmp_datas):
  cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

  resampling_method = 'SMOTETomek'

  df = pd.read_csv(data_choice+hmp_datas+'.csv', index_col=False).drop(['SampleID'], axis=1)
      
  #Get rid of Unknown IR_IS_classifications, encode IR as 0, IS as 1
  df.loc[:,'IR_IS_classification'] = (df.loc[:,'IR_IS_classification']).drop([i for i in range(len(df)) if df['IR_IS_classification'][i] == 'Unknown']
                                                                               , axis=0).replace({'IS':0, 'IR':1})
      
  #Remove blank entries
  remove_blanks = [i for i in range(len(df)) if df['Age'][i] == '' or df['BMI'][i] == '' or df['SSPG'][i] == '']
  df.loc[:,['Age', 'BMI', 'SSPG']] = df.loc[:,['Age', 'BMI', 'SSPG']].drop(remove_blanks, axis=0)

  #Convert Race to numbers
  is_asian = [int(df['Race'][i] == 'A') for i in range(len(df))]
  is_black = [int(df['Race'][i] == 'B') for i in range(len(df))]
  is_cauc = [int(df['Race'][i] == 'C') for i in range(len(df))]
  is_hisp = [int(df['Race'][i] == 'H') for i in range(len(df))]
  df['Asian?'], df['Black?'], df['Caucasian?'], df['Hispanic?'] = pd.DataFrame(is_asian), pd.DataFrame(is_black), pd.DataFrame(is_cauc), pd.DataFrame(is_hisp)

  #Convert Sex to numbers
  is_m = [int(df['Sex'][i] == 'M') for i in range(len(df))]
  is_f = [int(df['Sex'][i] == 'F') for i in range(len(df))]
  df['Male?'], df['Female?'] = pd.DataFrame(is_m), pd.DataFrame(is_f)
  df = df.drop(['Race', 'Sex'], axis=1).fillna(0)

  X = np.array(df.drop(['IR_IS_classification'], axis=1))
  y = np.array(df.loc[:,'IR_IS_classification'])
  scores = XGBModel(X, y, cv)
  metrics(scores, X, y, cv, resampling_method, data_choices[data_choice], hmp_datas)

In [6]:
hmp_datas = ['cytokine_abundance', 'gut_16s_abundance', 'Lipidomics', 
             'Metabolomics', 'nares_16s_abundance', 'Proteomics',
             'RNAseq_abundance', 'Targ.proteomics', 
             'Transcriptomics_VST_excl_3participants']

widgets = [' [',
        progressbar.Timer(format= 'elapsed time: %(elapsed)s'),
        '] ',
          progressbar.Bar('#'),' (',
          progressbar.ETA(), ') ',
          progressbar.Counter(format='%(value)d/%(max_value)d')
          ]
bar = progressbar.ProgressBar(max_value=6*len(hmp_datas), widgets=widgets).start()
count = 0

processes = list()
for i in data_choices:
  for j in range(len(hmp_datas)//2):
    p = multiprocessing.Process(target=run_model, args=(i, hmp_datas[j]))
    processes.append(p)
    p.start()

for p in processes:
  p.join()
  count += 1
  bar.update(count)

 [elapsed time: 0:02:07] |############                 | (ETA:   0:02:39) 24/54

In [7]:
processes.clear()
for i in data_choices:
  for j in range(len(hmp_datas)//2, len(hmp_datas)):
    p = multiprocessing.Process(target=run_model, args=(i, hmp_datas[j]))
    processes.append(p)
    p.start()

for p in processes:
  p.join()
  count += 1
  bar.update(count)

 [elapsed time: 0:21:28] |###########################  | (ETA:   0:00:49) 52/54